In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [161]:
# Leo los archivos a utilizar. Solo las columnas de mi interés.
trainData = pd.read_csv('../Data/train.csv', usecols=['id', 'text', 'target'])
testData = pd.read_csv('../Data/test.csv', usecols=['id', 'text'])

# Proceso el train y test set
Obtengo el train set como (word, target) y el test set como (id, word)

Me fijo cómo es la estructura de cada set de datos.

In [162]:
trainData.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [163]:
testData.tail()

,id,text
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,#CityofCalgary has activated its Municipal Eme...


## Proceso el train para obtener (word, target)
Es una función auxiliar que cambia la estructura del dataset de entrenamiendo de (id, text, target) a (word, target).

In [164]:
# Separo los textos de los tweets en palabras.
def splitTrainIntoWords(dataframe):
    splitted = pd.DataFrame(dataframe['text'].str.split(' ').to_list(), index=dataframe['id']).stack()
    
    # Reestablezco el id.
    splitted = splitted.reset_index([0, 'id'])

    # Le pongo a las segunda columna un nombre descriptivo.
    splitted.columns = ['id','word']

    # Agrego la columna target haciendo un merge con el set original.
    splitted = splitted.merge(dataframe, left_on = 'id', right_on = 'id', how = 'left')

    # Elimino la columna text y id.
    del splitted['text']
    del splitted['id']

    return splitted

## Proceso el test set para obtener (id, word)
Es una función auxiliar que cambia la estructura del dataset de test de (id, text) a (id, word).

In [165]:
# Separo los textos de los tweets en palabras.
def splitTestIntoWords(dataframe):
    # Creo un nuevo dataframe con las palabras separadas.
    splitted = pd.DataFrame(dataframe['text'].str.split(' ').to_list(), index=dataframe['id']).stack()
    
    # Reestablezco el id.
    splitted = splitted.reset_index([0, 'id'])

    # Le pongo a las segunda columna un nombre descriptivo.
    splitted.columns = ['id','word']

    return splitted

## Función de entrenamiento
Esta función toma el set de entrenamiento del formato (word, target) con el cuál se va a realizar el entrenamiento modelo y devuelve un dataframe entrenado del formato (word, target).

In [166]:
def entrenarModelo(train):    
    # Agrupo las palabras y hago un promedio del target.
    trainedDF = train.groupby('word').mean().reset_index()
    
    return trainedDF

## Función de predicción
Esta función realiza las predicciones.

Recibe un set entrenado con el formaro (word, target) y un set de test con el formato (id, word) sobre el cuál realiza las predicciones.

Devuelve un dataframe con las predicciones del tipo (id, predicción).

In [167]:
def predecir(trained, test, puntoLimite):    
    # Mergeo el valor del promedio del target de las palabras en este data set.
    testDF = test.merge(trained, left_on = 'word', right_on = 'word', how = 'left')
    
    # Elimino la columna de las palabras.
    del testDF['word']
    
    # Agrupo por id.
    testDF = testDF.groupby('id').mean().reset_index()
    
    # Paso los valores a 1 y 0 dependiendo si target >= puntoLimite o < puntoLimite.
    testDF['target'] = testDF['target'].apply(lambda x: 1 if x >= puntoLimite else 0)
    
    return testDF

## Función para crear submit.
Toma un set de datos de train con formaro (word, target), lo entrena, realiza las predicciones sobre el archivo test con formato (id, word) y guarda un archivo en caso de ser necesario.

In [168]:
def entrenarPredecirYGuardar(train, test, puntoLimite, csvFileName, save):
    trained = entrenarModelo(train)
    predicciones = predecir(trained, test, puntoLimite)  
    
    if save:
        predicciones.set_index('id').to_csv(csvFileName)
    
    return predicciones

# Creo funciones que van a ser las que se aplican a los datasets para realizar la optimización.

Para mejorar en cada iteración el resultado se crean diferentes funciones que hacen los siguientes cambios:

1 - Se pasan todas las palabras a lowercase.

2 - Se toman los links como si fueran una sola palabra 'http'.

3 - Se quitan los '\n' encontrados en los textos.

4 - Se quitan los stop words.

5 - Se quitan de las palabras todos los caracteres que no sean letras y se eliminan las palabras vacías.

En cada paso se hará un nuevo submit verificando los resultados obtenidos.

## Nomenclatura de los archivos
Se tienen 5 funcionas a aplicar a los datasets, entonces los archivos se nombraran como finger-xxxxx.csv.

Si solo se aplica la función 1 entonces el archivo sera finger-00001.csv, si no se aplica ninguna función será finger-00000.csv y si se aplican la función 1 y la función 3 será finger-00101.csv.

## Función para pasar las palabras a minúsculas
Recibe un set de datos con una columna 'word' y devuelve un nuevo set con las palabras en minúscula.

In [169]:
def palabrasALowercase(trained):
    lowercasedDF = trained.copy()
    
    # Paso las palabras a minúsculas.
    lowercasedDF['word'] = lowercasedDF['word'].str.lower()
    
    return lowercasedDF

## Función para tomar los links como una única palabra 'http'
Recibe un set de datos con la columna 'word' y devuelve un nuevo set con los links como si fueran una misma palabra 'http'.

In [170]:
def linksComoHttp(trained):
    httpDF = trained.copy()
    
    # Todos los links pasan a ser la palabra 'http'.
    httpDF.loc[httpDF['word'].str.contains('http', case=True), 'word'] = 'http'
    
    return httpDF

## Función que quita los saltos de línea '\n'
Recibe un set de datos entrenados, un set de test del formato (word, target) o (id, word) y un train set o test set con los la etiquera '\n' eliminada.

In [171]:
def eliminarEtiquetasTrain(train):
    trainDF = train.copy()
    trainDF = pd.DataFrame(trainDF['word'].str.split('\n').to_list(), index=trainDF['target']).stack()
    trainDF = trainDF.reset_index([0, 'target'])
    trainDF.columns = ['target','word']
    trainDF.reset_index()
    
    return trainDF

def eliminarEtiquetasTest(test):
    testDF = test.copy()
    testDF = pd.DataFrame(testDF['word'].str.split('\n').to_list(), index=testDF['id']).stack()
    testDF = testDF.reset_index([0, 'id'])
    testDF.columns = ['id','word']
    testDF.reset_index()
    
    return testDF

## Función que quita todos los caracteres que no sean letras de las palabras.
Dado un dataframe con la columna 'word', elimina caracteres especiales y borra todas las palabras que sean vacías.

In [172]:
def eliminarCharsEspeciales(dataframe): 
    sanDF = dataframe.copy()
    sanDF = sanDF[sanDF['word'].str.isspace() == False]
    sanDF['word'].replace(regex=True, inplace=True, to_replace='[^A-Za-z]', value=r'')
    sanDF = sanDF[sanDF['word'].str.strip().astype(bool)]
    
    return sanDF

## Función para quitar las stop words.
Dado un dataframe con la columna 'word', elimina todos los stop words.

In [173]:
def eliminarStopWords(dataframe):
    stops = stopwords.words('english')
    sinStops = dataframe.copy()
    sinStops = sinStops[~sinStops['word'].isin(stops)]
    return sinStops

# Submits.
A continuación se realizan los submits de prueba.

In [174]:
# 00000.
trainSplitted = splitTrainIntoWords(trainData)
testSplitted = splitTestIntoWords(testData)
entrenarPredecirYGuardar(trainSplitted, testSplitted, 0.5, 'finger-00000.csv', True)['target'].value_counts()

0    2281
1     982
Name: target, dtype: int64

In [175]:
# 00001.
# Se aplica lowercase a las palabras de los dataframes.
trainLowercased = palabrasALowercase(trainSplitted)
testLowercase = palabrasALowercase(testSplitted)
entrenarPredecirYGuardar(trainLowercased, testLowercase, 0.5, 'finger-00001.csv', True)['target'].value_counts()

0    2293
1     970
Name: target, dtype: int64

In [176]:
# 00010.
# Se toman los links como la palabra 'http'.
trainHTTP = linksComoHttp(trainSplitted)
testHTTP = linksComoHttp(testSplitted)
entrenarPredecirYGuardar(trainHTTP, testHTTP, 0.5, 'finger-00010.csv', True)['target'].value_counts()

0    2267
1     996
Name: target, dtype: int64

In [177]:
# 00100.
# Se eliminan las etiquetas como '\n'.
trainSinEtiquetas = eliminarEtiquetasTrain(trainSplitted)
testSinEtiquetas = eliminarEtiquetasTest(testSplitted)
entrenarPredecirYGuardar(trainSinEtiquetas, testSinEtiquetas, 0.5, 'finger-00100.csv', True)['target'].value_counts()

0    2276
1     987
Name: target, dtype: int64

In [178]:
# 01000.
# Se eliminan las stopwords.
trainStops = eliminarStopWords(trainSplitted)
testStops = eliminarStopWords(testSplitted)

entrenarPredecirYGuardar(trainStops, testStops, 0.5, 'finger-01000.csv', True)['target'].value_counts()

0    2147
1    1116
Name: target, dtype: int64

In [179]:
# 10000.
# Se eliminan los caracteres especiales y las palabras vacías..
trainSan = eliminarCharsEspeciales(trainSplitted)
testSan = eliminarCharsEspeciales(testSplitted)
entrenarPredecirYGuardar(trainSan, testSan, 0.5, 'finger-10000.csv', True)['target'].value_counts()

0    2305
1     958
Name: target, dtype: int64

# Grid Search + Cross validation.
A continuación se intentaran buscar cuál es la combinación de funciones que mejores predicciones realiza.<br>
La idea es buscar, dentro de las 5 funciones declaradas, las funciones que minimizan los errores de predicción. <br>
Para cada combinación de funciones se dividirá el set de datos en 10 subsets, se entrenara el algoritmo con 9 subsets y se utilizará el subset restante como set de validación. Este proceso se realizará 10 veces con el fin de que cada subset actue como set de validación.<br>
Una vez que se corrieron las 10 iteraciones para cada combinación de funciones, se tomara como resultado el promedio de los valores obtenidos para cada iteración y se elige aquela combinación que mejor resultado logró.

## Función que aplica las funciones declaradas 
Esta función lo que hace es recibir una lista de enteros que indican si la función se debe aplicar o no dependiendo si en la posición se encuentra un 1 o un 0. Por ejemplo: si se recibe la lista [0, 1, 1, 0, 0] esto indica lo siguiente: <br>
0 -> No se aplica la función 5.<br>
1 -> Se aplica la función 4.<br>
1 -> Se aplica la función 3.<br>
0 -> No se aplica la función 2.<br>
0 -> No se aplica la función 1.<br>

El set de entrenamiento y el de test tienen que tener los siguientes formatos: (word, target) y (id, word) respectivamente.

In [ ]:
def aplicarFunciones(train, test, funciones):
    trainSet = train.copy()
    testSet = test.copy()
    
    if funciones[4]:
        trainSet = palabrasALowercase(trainSet)
        testSet = palabrasALowercase(testSet)
        
    if funciones[3]:
        trainSet = linksComoHttp(trainSet)
        testSet = linksComoHttp(testSet)
        
    if funciones[2]:
        trainSet = eliminarEtiquetasTrain(trainSet)
        testSet = eliminarEtiquetasTest(testSet)
        
    if funciones[1]:
        trainSet = eliminarStopWords(trainSet)
        testSet = eliminarStopWords(testSet)
        
    if funciones[0]:
        trainSet = eliminarCharsEspeciales(trainSet)
        testSet = eliminarCharsEspeciales(testSet)
        
    return trainSet, testSet

## Función de evaluación
Esta función recibe un dataframe de predicciones (id, target) con su dataframe de validación (id, target). <br>
Simplemente valida si se realizó bien o mal la predicción y, mediante los positivos verdaderos, los falsos positivos y falsos negativos, calcula la función F1 de evaluación.

In [ ]:
def evaluarPredicciones(predicciones, validaciones):
    verdaderoPositivo = 0
    falsoPositivo = 0
    falsoNegativo = 0
    
    for i in range(0, predicciones.shape[0] - 1):
        prediccion = predicciones.iloc[i, :]
        idPrediccion = prediccion['id']
        valorReal = validaciones[validaciones['id'] == idPrediccion].iloc[0,:]

        prediccionTarget = prediccion['target']
        valorRealTarget = valorReal['target']
    
        if prediccionTarget == 1 and valorRealTarget == 1:
            verdaderoPositivo = verdaderoPositivo + 1
            
        if prediccionTarget == 1 and valorRealTarget == 0:
            falsoPositivo = falsoPositivo + 1
                
        if prediccionTarget == 0 and valorRealTarget == 1:
            falsoNegativo = falsoNegativo + 1
    
    if verdaderoPositivo + falsoPositivo == 0:
        return 0
            
    precision = verdaderoPositivo / (verdaderoPositivo + falsoPositivo)
    recall = verdaderoPositivo / (verdaderoPositivo + falsoNegativo)
    
    return 2 * ((precision * recall) / (precision + recall))

## Función cross validation
Esta función realiza el proceso de validación para la combinación de funciones recibida. <br>
Dado el set de entrenamiento (id, word, target) y el porcentage por el cuál dividir el train divide el set de entrenamiento en dos sub set: set de entrenamiento (uno nuevo, de menor tamaño) y set de validación. <br>
A partir del set de validación genera un set de testeo, sobre el cuál se van a realizar las predicciones. <br>
Se entrena el modelo con el subset de entrenamiento y se realizan las predicciones sobre el set de testeo. <br>
Se itera hasta que todos los subsets que se pueden armar con el porcentage de subsets pasado hayan sido utilizados como subset de validación. <br>
El resultado final del cross validation es el promedio del resultado de todas las iteraciones realizadas.

In [ ]:
def crossValidation(train, subSetPercentage, funciones, puntoLimite):
    # La cantidad máxima de observaciones de un sub set.
    subSetMaxSize = round(train.shape[0] * subSetPercentage) + 1
    
    # Los índices para el primer set de validación.
    subSetStartIndex = 0
    subSetEndIndex = subSetMaxSize
    
    # El máximo index de mi dataframe.
    setMaxIndex = train.shape[0]
    iteraciones = 0
    acumulador = 0

    while subSetStartIndex <= setMaxIndex:        
        # Armo el set de entrenamiento y el set de validación.
        validationSet = train[subSetStartIndex:subSetEndIndex]
        trainSet = train.drop(train.index[subSetStartIndex:subSetEndIndex])
        
        # Empiezo el entrenamiento.
        trainSet = splitTrainIntoWords(trainSet)
        testSet = splitTestIntoWords(validationSet)
        
        # Aplico las funciones correspondientes al set de entrenamiento y de test.
        trainSet, testSet = aplicarFunciones(trainSet, testSet, funciones)
            
        # Realizo las predicciones.
        predicciones = entrenarPredecirYGuardar(trainSet, testSet, puntoLimite, '', False)
        puntaje = evaluarPredicciones(predicciones, validationSet)
        
        # Aumento los valores para una nueva iteración.
        subSetStartIndex = subSetEndIndex + 1
        subSetEndIndex = subSetStartIndex + subSetMaxSize
        iteraciones = iteraciones + 1
    
        acumulador = acumulador + puntaje
       
    # Se devuelve el promedio de los puntajes obtenidos.
    return acumulador / iteraciones

## Función grid search.
En esta función lo que se hace es bucar la combinación de funciones que mejor resultado dan al predecir el test de entrenamiento mediante cross validation. <br>

In [ ]:
def gridSearch(train):
    # Grid search sobre todas las posibilidades para ver cuál ajusta mejor.
    cantidadDeFunciones = 5
    mejorResultado = 0
    mejorCombinacion = [0, 0, 0, 0, 0]
    puntoLimite = 0.0
    puntoLimiteVar = 0.1
    mejorPuntoLimite = 0.0
    
    while puntoLimite <= 1:
        print('############################################################################################################')
        print('Punto Límite: ', puntoLimite)
        print('############################################################################################################')

        for number in range(pow(2, cantidadDeFunciones)):
            bits = [(number >> bit) & 1 for bit in range(cantidadDeFunciones - 1, -1, -1)]
            
            validacion = crossValidation(train, 0.1, bits, puntoLimite)
            
            if validacion > mejorResultado:
                mejorResultado = validacion
                mejorCombinacion = bits
                mejorPuntoLimite = puntoLimite
            
            print('Comb: ', bits, '; Res: ', validacion, '; Mejor Res: ', mejorResultado, '; Mejor Comb: ', mejorCombinacion)
        
        puntoLimite = puntoLimite + puntoLimiteVar
    
    return mejorResultado, mejorCombinacion, puntoLimite

A continuación se busca la mejor combinación mediante el gridSearch + cross validation.

In [ ]:
train = trainData.copy()
resultado, combinacion, puntoLimite = gridSearch(train)

Una vez que se encontró la mejor combinación se usa la combinación para realizar predicciones sobre el set de test.

In [185]:
# Creo el nombre del archivo en el que se guardará el resultado.
nombreArchivo = 'finger-best.csv'

In [188]:
# Se crea el train y el set con la combinación que dió mejor resultado.
mejorTrain = splitTrainIntoWords(trainData)
mejorTest = splitTestIntoWords(testData)
mejorTrain, mejorTest = aplicarFunciones(mejorTrain, mejorTest, [1,1,1,0,1])

In [189]:
# Se realiza la predicción y se guarda el archivo.
entrenarPredecirYGuardar(mejorTrain, mejorTest, 0.5, nombreArchivo, True)['target'].value_counts()

0    2137
1    1126
Name: target, dtype: int64